# Capacitated Vehicle Routing Problem with Time Window (CVRPTW)

In [4]:
import sys
from collections import defaultdict
import xlrd
import gurobipy as gp
from gurobipy import GRB
sys.path.insert(1, '/home/AzureUser/notebooks/agci-logistics/Routing/NewRoutingCore/src/OptimizerAlgorithm')

In [14]:
from all_functions import parseXML, logistic_core
import glob
import pandas as pd
import os
import time
import json

In [105]:
class Technician():
    def __init__(self, name, cap, maxwt, depot):
        self.name = name
        self.cap = cap
        self.maxwt = maxwt
        self.depot = depot

    def __str__(self):
        return f"Technician: {self.name}\n  Capacity: {self.cap}\n  MaxWt: {self.maxwt}\n  Depot: {self.depot}"
    
class Job():
    def __init__(self, name, priority, duration, coveredBy):
        self.name = name
        self.priority = priority
        self.duration = duration
        self.coveredBy = coveredBy

    def __str__(self):
        about = f"Job: {self.name}\n  Priority: {self.priority}\n  Duration: {self.duration}\n  Covered by: "
        about += ", ".join([t.name for t in self.coveredBy])
        return about
    
class Customer():
    def __init__(self, name, loc, job, quantity, tStart, tEnd, tDue):
        self.name = name
        self.loc = loc
        self.job = job
        self.quantity = quantity
        self.tStart = tStart
        self.tEnd = tEnd
        self.tDue = tDue

    def __str__(self):
        coveredBy = ", ".join([t.name for t in self.job.coveredBy])
        return f"Customer: {self.name}\n  Location: {self.loc}\n  Job: {self.job.name}\n  Quantity: {self.quantity}\n  Priority: {self.job.priority}\n  Duration: {self.job.duration}\n  Covered by: {coveredBy}\n  Start time: {self.tStart}\n  End time: {self.tEnd}\n  Due time: {self.tDue}"


In [133]:
def solve_trs0(technicians, customers, dist, params):
    # Build useful data structures
    K = [k.name for k in technicians]
    C = [j.name for j in customers]
    J = [j.loc for j in customers]
    L = list(set([l[0] for l in dist.keys()]))
    D = list(set([t.depot for t in technicians]))
    cap = {k.name : k.cap for k in technicians}
    wts = {k.name : k.maxwt for k in technicians}  # new
    loc = {j.name : j.loc for j in customers}
    depot = {k.name : k.depot for k in technicians}
    canCover = {j.name : [k.name for k in j.job.coveredBy] for j in customers}
    dur = {j.name : j.job.duration for j in customers}
    tStart = {j.name : j.tStart for j in customers}
    tEnd = {j.name : j.tEnd for j in customers}
    tDue = {j.name : j.tDue for j in customers}
    priority = {j.name : j.job.priority for j in customers}
    quantity = {j.loc : j.quantity for j in customers}  # new
    quantity['Depot'] = 0
    
        ### Create model
    m = gp.Model("trs0")
    
    ### Decision variables
    # Customer-technician assignment
    x = m.addVars(C, K, vtype=GRB.BINARY, name="x")
    
    # Technician assignment
    u = m.addVars(K, vtype=GRB.BINARY, name="u")
    
    # Edge-route assignment to technician
    y = m.addVars(L, L, K, vtype=GRB.BINARY, name="y")
   
    # Technician cannot leave or return to a depot that is not its base
    for k in technicians:
        for d in D:
            if k.depot != d:
                for i in L:
                    y[i,d,k.name].ub = 0
                    y[d,i,k.name].ub = 0
    
    # Start time of service
    t = m.addVars(L, ub=600, name="t")
    
    # Lateness of service
    z = m.addVars(C, name="z")
    
    # Artificial variables to correct time window upper and lower limits
    xa = m.addVars(C, name="xa")
    xb = m.addVars(C, name="xb")
    
    # Unfilled jobs
    g = m.addVars(C, vtype=GRB.BINARY, name="g")
    
    ### Constraints

    # A technician must be assigned to a job, or a gap is declared (1)
    m.addConstrs((gp.quicksum(x[j, k] for k in canCover[j]) + g[j] == 1 for j in C), name="assignToJob")
    
    # At most one technician can be assigned to a job (2)
    m.addConstrs((x.sum(j, '*') <= 1 for j in C), name="assignOne")

    # Technician capacity constraints (3)
    capLHS = {k : gp.quicksum(dur[j]*x[j,k] for j in C) +\
        gp.quicksum(dist[i,j]*y[i,j,k] for i in L for j in L) for k in K}
    m.addConstrs((capLHS[k] <= cap[k]*u[k] for k in K), name="techCapacity")

    # Technician tour constraints (4 and 5)
    m.addConstrs((y.sum('*', loc[j], k) == x[j,k] for k in K for j in C),\
        name="techTour1")
    m.addConstrs((y.sum(loc[j], '*', k) == x[j,k] for k in K for j in C),\
        name="techTour2")

    # Same depot constraints (6 and 7)
    m.addConstrs((gp.quicksum(y[j,depot[k],k] for j in J) == u[k] for k in K),\
        name="sameDepot1")
    m.addConstrs((gp.quicksum(y[depot[k],j,k] for j in J) == u[k] for k in K),\
        name="sameDepot2")
    
    # weight constraints
    wtLHS = {k : gp.quicksum(quantity[j]*y[i,j,k] for i in L for j in L) for k in K}
    m.addConstrs((wtLHS[k] <= wts[k] for k in K), 'weightCapacity')

    # Temporal constraints (8) for customer locations
    M = {(i,j) : 600 + dur[i] + dist[loc[i], loc[j]] for i in C for j in C}
    m.addConstrs((t[loc[j]] >= t[loc[i]] + dur[i] + dist[loc[i], loc[j]]\
        - M[i,j]*(1 - gp.quicksum(y[loc[i],loc[j],k] for k in K))\
        for i in C for j in C), name="tempoCustomer")

    # Temporal constraints (8) for depot locations
    M = {(i,j) : 600 + dist[i, loc[j]] for i in D for j in C}
    m.addConstrs((t[loc[j]] >= t[i] + dist[i, loc[j]]\
        - M[i,j]*(1 - y.sum(i,loc[j],'*')) for i in D for j in C),\
        name="tempoDepot")

    # Time window constraints (9 and 10)
    m.addConstrs((t[loc[j]] + xa[j] >= tStart[j] for j in C), name="timeWinA")
    m.addConstrs((t[loc[j]] - xb[j] <= tEnd[j] for j in C), name="timeWinB")

    # Lateness constraint (11)
    m.addConstrs((z[j] >= t[loc[j]] + dur[j] - tDue[j] for j in C),\
        name="lateness")

    ### Objective function
    M = 6100
    
    # total Trucks used
    trucksUsed = {k: gp.quicksum(y[i,j,k] for i in L for j in L) for k in K}
#     trucksUsedBin = {k: (1 if trucksUsed[k] > 0 else 0) for k in K}
    nbTrucksUsed = gp.quicksum(trucksUsed[k] for k in K)

    # total distance
    totaldist = gp.quicksum(y[i,j,k]*dist[i,j] for i in L for j in L for k in K)

    if params['multiObjective']:
        m.ModelSense = GRB.MINIMIZE
        # Multi-objective
        m.setObjectiveN(z.prod(priority) + gp.quicksum( 0.01 * M * priority[j] * (xa[j] + xb[j]) for j in C) +
                       gp.quicksum( M * priority[j] * g[j] for j in C), index=0, priority=3)
        m.setObjectiveN(totaldist, index=1, priority=2)
#         m.setObjectiveN(nbTrucksUsed, index=2, priority=1)
    else:
        # single objective
        m.setObjective(z.prod(priority) + gp.quicksum( 0.01 * M * priority[j] * (xa[j] + xb[j]) for j in C) +
                   gp.quicksum( M * priority[j] * g[j] for j in C) + totaldist + nbTrucksUsed, GRB.MINIMIZE)
    
    m.write("TRS0.lp")
    
    # set parameters
    m.setParam("TimeLimit", params["TimeLimit"])
    m.optimize()

    status = m.Status
    if status in [GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED]:
        print("Model is either infeasible or unbounded.")
        sys.exit(0)
    elif status != GRB.OPTIMAL:
        print("Optimization terminated with status {}".format(status))
#         sys.exit(0)
        
    ### Print results
    # Assignments
    print("")
    for j in customers:
        if g[j.name].X > 0.5:
            jobStr = "Nobody assigned to {} ({}) in {}".format(j.name,j.job.name,j.loc)
        else:
            for k in K:
                if x[j.name,k].X > 0.5:
                    jobStr = "{} assigned to {} ({}) in {}. Start at t={:.2f}.".format(k,j.name,j.job.name,j.loc,t[j.loc].X)
                    if z[j.name].X > 1e-6:
                        jobStr += " {:.2f} minutes late.".format(z[j.name].X)
                    if xa[j.name].X > 1e-6:
                        jobStr += " Start time corrected by {:.2f} minutes.".format(xa[j.name].X)
                    if xb[j.name].X > 1e-6:
                        jobStr += " End time corrected by {:.2f} minutes.".format(xb[j.name].X)
        print(jobStr)

    # Technicians
    print("")
    for k in technicians:
        if u[k.name].X > 0.5:
            cur = k.depot
            route = k.depot
            cum_dist = 0
            while True:
                for j in customers:
                    if y[cur,j.loc,k.name].X > 0.5:
                        route += " -> {} (dist={}, t={:.2f}, proc={}, req={})".format(j.loc, dist[cur,j.loc], t[j.loc].X, j.job.duration, j.quantity)
                        cum_dist += dist[cur,j.loc]
                        cur = j.loc
                for i in D:
                    if y[cur,i,k.name].X > 0.5:
                        route += " -> {} (dist={})".format(i, dist[cur,i])
                        cum_dist += dist[cur,i]
                        cur = i
                        break
                if cur == k.depot:
                    break
            print("{}'s route: {} (total distance {})".format(k.name, route, cum_dist))
        else:
            print("{} is not used".format(k.name)) 
            
    
    # Utilization
    print("")
    for k in K:
        used = capLHS[k].getValue()
        total = cap[k]
        util = used / cap[k] if cap[k] > 0 else 0
        print("{}'s utilization is {:.2%} ({:.2f}/{:.2f})".format(k, util,\
            used, cap[k]))
    totUsed = sum(capLHS[k].getValue() for k in K)
    totCap = sum(cap[k] for k in K)
    totUtil = totUsed / totCap if totCap > 0 else 0
    print("Total technician utilization is {:.2%} ({:.2f}/{:.2f})".format(totUtil, totUsed, totCap))
    
    
    

In [67]:
def printScen(scenStr):
    sLen = len(scenStr)
    print("\n" + "*"*sLen + "\n" + scenStr + "\n" + "*"*sLen + "\n")


In [108]:
def read_data(path_base, problem='test'):

    path_to_distance = os.path.join(path_base,"Distances." + problem + ".json")
    path_to_travel_time = os.path.join (path_base,"TravelTimes." + problem + ".json")
    path_to_fleet = os.path.join (path_base,"Fleet." + problem + ".json")
    path_to_requests = os.path.join (path_base,"Requests." + problem + ".json")

    # Read the distance file
    with open(path_to_distance) as fp:
        A = json.load(fp)
        distList = A['values']
    
    n = len(distList)
    nodes = ['Depot'] + ['L'+str(i) for i in range(1,n)]
    dist = {(nodes[i], nodes[j]): distList[i][j] for i in range(n) for j in range(n)}

    with open(path_to_travel_time) as fp:
        # INCONSISTENCY: This does not work for the benchmark instances
        # self.time = json.load(fp)[]
        tdata = json.load(fp)
    travelTimes = {(nodes[i], nodes[j]): tdata[i][j] for i in range(n) for j in range(n)}

    with open(path_to_fleet) as fp:
        A = json.load(fp)
        vmodel = A['models']

    technicians = []
    for i,t in enumerate(vmodel):
        # Create Technician object
        numVehicle = t['number']
        for j in range(numVehicle):
            # depot = t['starting_node']
            thisTech = Technician(name='T'+str(i)+str(j), cap=t['ending_time'], maxwt=t['max_weight'], depot='Depot')
            technicians.append(thisTech)
    print(f"Total {len(technicians)} vehicles")
                
    # Read job data, only one type
    jobs = []
    coveredBy = [t for i,t in enumerate(technicians)]
    # Create Job object, name, priority, duration, coveredBy
    thisJob = Job(name='J1', priority=4, duration=1, coveredBy=coveredBy)
    jobs.append(thisJob)

    with open(path_to_requests) as fp:
        A = json.load(fp)
        request = list(A)

    customers = []
    for i,r in enumerate(request):
        # name, loc, job, quantity, tStart, tEnd, tDue
        thisCustomer = Customer(name='C'+r['name'], loc='L'+r['name'], job=jobs[0], 
                                quantity=r['quantity'], tStart=r['earliest_unload_time'], 
                                tEnd=r['latest_unload_time'], tDue=r['latest_unload_time'])
        customers.append(thisCustomer)

    return technicians, customers, dist, travelTimes

In [96]:
# datafiles = glob.glob("/home/AzureUser/notebooks/Routing_data/C2/*.xml")
datafiles = ["/home/AzureUser/notebooks/Routing_data/C2/C2_02_01.xml"]
dest_dir = '/home/AzureUser/notebooks/gurobi-models/modeling-examples/technician_routing_scheduling/test'
dir_google = '/home/AzureUser/notebooks/agci-logistics/Routing/NewRoutingCore/data/all_dataORToolsCVRP_16_0_v2'

In [9]:
source = datafiles[0]
filename = source.split('/')[-1]
nbCustomers = int(filename.split('_')[1])*100
problem = filename.split('.xml')[0]
all_files = parseXML(source, data_dir=dest_dir)    

Read 201 coordinates and 1 vehicle type ..


In [135]:
if __name__ == "__main__":
    
#     technicians, customers, dist, ttime = read_data(dest_dir)
    technicians, customers, dist, ttime = read_data(dir_google, 'ORToolsCVRP_16_0')
    # Base model
    printScen("Solving base scenario model")
    solve_trs0(technicians, customers, dist, {"TimeLimit": 3600, 'multiObjective': False})


Total 5 vehicles

***************************
Solving base scenario model
***************************

Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 532 rows, 1611 columns and 7999 nonzeros
Model fingerprint: 0x5a9e0831
Variable types: 65 continuous, 1546 integer (1546 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+00, 2e+04]
  Bounds range     [1e+00, 6e+02]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective 390400.00000
Presolve removed 409 rows and 1349 columns
Presolve time: 0.03s
Presolved: 123 rows, 262 columns, 1197 nonzeros
Variable types: 7 continuous, 255 integer (255 binary)

Root relaxation: objective 1.035509e+05, 160 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Ti

In [107]:
for t in technicians:
    print(t)

Technician: T00
  Capacity: 15.0
  MaxWt: 15.0
  Depot: Depot
Technician: T01
  Capacity: 15.0
  MaxWt: 15.0
  Depot: Depot
Technician: T02
  Capacity: 15.0
  MaxWt: 15.0
  Depot: Depot
Technician: T03
  Capacity: 15.0
  MaxWt: 15.0
  Depot: Depot
Technician: T04
  Capacity: 15.0
  MaxWt: 15.0
  Depot: Depot


In [103]:
ttime

{('Depot', 'Depot'): 0,
 ('Depot', 'L1'): 548,
 ('Depot', 'L2'): 776,
 ('Depot', 'L3'): 696,
 ('Depot', 'L4'): 582,
 ('Depot', 'L5'): 274,
 ('Depot', 'L6'): 502,
 ('Depot', 'L7'): 194,
 ('Depot', 'L8'): 308,
 ('Depot', 'L9'): 194,
 ('Depot', 'L10'): 536,
 ('Depot', 'L11'): 502,
 ('Depot', 'L12'): 388,
 ('Depot', 'L13'): 354,
 ('Depot', 'L14'): 468,
 ('Depot', 'L15'): 776,
 ('Depot', 'L16'): 662,
 ('L1', 'Depot'): 548,
 ('L1', 'L1'): 0,
 ('L1', 'L2'): 684,
 ('L1', 'L3'): 308,
 ('L1', 'L4'): 194,
 ('L1', 'L5'): 502,
 ('L1', 'L6'): 730,
 ('L1', 'L7'): 354,
 ('L1', 'L8'): 696,
 ('L1', 'L9'): 742,
 ('L1', 'L10'): 1084,
 ('L1', 'L11'): 594,
 ('L1', 'L12'): 480,
 ('L1', 'L13'): 674,
 ('L1', 'L14'): 1016,
 ('L1', 'L15'): 868,
 ('L1', 'L16'): 1210,
 ('L2', 'Depot'): 776,
 ('L2', 'L1'): 684,
 ('L2', 'L2'): 0,
 ('L2', 'L3'): 992,
 ('L2', 'L4'): 878,
 ('L2', 'L5'): 502,
 ('L2', 'L6'): 274,
 ('L2', 'L7'): 810,
 ('L2', 'L8'): 468,
 ('L2', 'L9'): 742,
 ('L2', 'L10'): 400,
 ('L2', 'L11'): 1278,
 ('L2'

In [104]:
for c in customers:
    print(c)

Customer: C1
  Location: L1
  Job: J1
  Quantity: 1.0
  Priority: 4
  Duration: 1
  Covered by: T00, T01, T02, T03, T04
  Start time: 0.0
  End time: 1000000.0
  Due time: 1000000.0
Customer: C2
  Location: L2
  Job: J1
  Quantity: 1.0
  Priority: 4
  Duration: 1
  Covered by: T00, T01, T02, T03, T04
  Start time: 0.0
  End time: 1000000.0
  Due time: 1000000.0
Customer: C3
  Location: L3
  Job: J1
  Quantity: 2.0
  Priority: 4
  Duration: 1
  Covered by: T00, T01, T02, T03, T04
  Start time: 0.0
  End time: 1000000.0
  Due time: 1000000.0
Customer: C4
  Location: L4
  Job: J1
  Quantity: 4.0
  Priority: 4
  Duration: 1
  Covered by: T00, T01, T02, T03, T04
  Start time: 0.0
  End time: 1000000.0
  Due time: 1000000.0
Customer: C5
  Location: L5
  Job: J1
  Quantity: 2.0
  Priority: 4
  Duration: 1
  Covered by: T00, T01, T02, T03, T04
  Start time: 0.0
  End time: 1000000.0
  Due time: 1000000.0
Customer: C6
  Location: L6
  Job: J1
  Quantity: 4.0
  Priority: 4
  Duration: 1
  Covere